In [1]:
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, ConstantKernel
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
import os
import shutil

In [15]:
mm1 = 200
mm2 = 200
a0 = 0.0
a1 = -1.0
b0 = 0.3
b1 = 0.5
b = np.arange(a0, a1, (a1-a0)/mm1)
d = np.arange(b0, b1, (b1-b0)/mm2)
B, D = np.meshgrid(b, d)
n, m = B.shape

In [17]:
nuarray = np.zeros((20, 200, 200))
for jter in range(20):
    fname_accum = '/Users/user/Dropbox/실험일지/2025 실험/4K JSLee/20250207 acuum(iter=20)/accum_'+str(jter)+'.npy'
    #Info.fname_accum = fname_accum
    if True:
        #gname = Info.fname_accum[:-4] + '.npy'
        with open(fname_accum, 'rb') as afile:
            nu = np.load(afile)
            st = np.load(afile)
        nuarray[jter, :, :] = nu[:, :]

In [19]:
etam = np.zeros(20)
for jter in range(20):
    fname_accum = '/Users/user/Dropbox/실험일지/2025 실험/4K JSLee/20250207 acuum(iter=20)/accum_'+str(jter)+'.txt'
    x = []
    y = []
    z = []
    afile = open(fname_accum, 'r')
    for line in afile:
        x.append(float(line.split()[0]))
        y.append(float(line.split()[1]))
        z.append(float(line.split()[2]))
    afile.close()
    #Info.fname_accum = fname_accum
    etam[jter] = 0.
    count = 0.
    for k in range(len(z)):
        for i0 in range(200):
            if np.abs(x[k]-b[i0]) < 1e-3:
                for j0 in range(200):
                    if np.abs(y[k]-d[j0]) < 1e-3:
                        etam[jter] = etam[jter] + (z[k]-nuarray[0, i0, j0])**2
                        count = count+1.

KeyboardInterrupt: 

In [ ]:
np.sqrt(np.sum(etam))/count/20.

In [24]:
etap = np.zeros(20)
for jter in range(20):
    fname_accum = '/Users/user/Dropbox/실험일지/2025 실험/4K JSLee/20250207 acuum(iter=20)/accum_'+str(jter)+'.txt'
    #Info.fname_accum = fname_accum
    etap[jter] = 0.
    count = 0.
    for i0 in range(200):
        for j0 in range(200):
            etap[jter] = etap[jter] + (nuarray[jter, i0, j0]-nuarray[0, i0, j0])**2
            count = count+1.

In [26]:
np.sqrt(np.sum(etap))/count/20.

np.float64(5.637089469457365e-05)

In [ ]:
nexp = 3
xyz = np.zeros((nexp, 200, 200))
for jter in range(nexp):
    fname_accum = 'data_iv_' + str(jter+1) + '.txt'
    x = []
    y = []
    z = []
    afile = open(fname_accum, 'r')
    for line in afile:
        y.append(float(line.split()[0]))
        x.append(float(line.split()[1]))
        z.append(float(line.split()[2]))
    afile.close()
    z = np.array(z)
    x = np.array(x)
    y = np.array(y)
    xx = np.zeros((200, 200))
    yy = np.zeros((200, 200))
    zz = np.zeros((200, 200))
    k = 0
    for i in range(200):
        for j in range(200):
            xx[i, j] = x[k]
            yy[i, j] = y[k]
            zz[i, j] = z[k]
            k = k+1
    if False:
        print(len(z))
    xyz[jter, :, :] = xx[:, :]
    xyz[jter, :, :] = yy[:, :]
    xyz[jter, :, :] = zz[:, :]

In [ ]:
zavgexp = np.zeros((200, 200))
for i in range(200):
    for j in range(200):
        zavgexp[i, j] = sum(xyz[:, i, j])/nexp

In [ ]:
tmp = np.zeros((nexp, 200, 200))
for iexp in range(nexp):
    tmp[iexp, :, :] = ((xyz[iexp, :, :]-zavgexp[:, :])/zavgexp[:, :])**2

In [ ]:
tmp = np.zeros((nexp, 200, 200))
for iexp in range(nexp):
    tmp[iexp, :, :] = ((xyz[iexp, :, :]-zavgexp[:, :]))**2

In [ ]:
savgexp = np.zeros((200, 200))
for i in range(200):
    for j in range(200):
        savgexp[i, j] = np.sqrt(sum(tmp[:, i, j])/nexp)

In [ ]:
nuarray = np.zeros((nexp, 200, 200))
for iexp in range(nexp):
    fname_accum = r'C:\Users\Inho Lee\testAI\data_kn(ts=0.3)_20250211'+'/accum_'+str(jter)+'.npy'
    with open(fname_accum, 'rb') as afile:
        nu = np.load(afile)
        st = np.load(afile)
        nuarray[iexp, :, :] = nu[:, :]

In [ ]:
nuavexp = np.zeros((200, 200))
for i in range(200):
    for j in range(200):
        nuavexp[i, j] = sum(nuarray[:, i, j])/nexp

In [ ]:
tmp = np.zeros((nexp, 200, 200))
for iexp in range(nexp):
    tmp[iexp, :, :] = ((nuarray[iexp, :, :]-zavgexp[:, :])/zavgexp[:, :])**2

In [ ]:
tmp = np.zeros((nexp, 200, 200))
for iexp in range(nexp):
    tmp[iexp, :, :] = ((nuarray[iexp, :, :]-zavgexp[:, :]))**2

In [ ]:
spm = np.zeros((200, 200))
for i in range(200):
    for j in range(200):
        spm[i, j] = np.sqrt(
            sum(((tmp[:, i, j]-zavgexp[i, j])/zavgexp[i, j])**2)/nexp)

In [ ]:
spm = np.zeros((200, 200))
for i in range(200):
    for j in range(200):
        spm[i, j] = np.sqrt(
            sum(((tmp[:, i, j]-zavgexp[i, j]))**2)/nexp)

In [ ]:
print(spm)

In [ ]:
asum = 0.
for i in range(200):
    for j in range(200):
        asum = asum+spm[i, j]
print(asum/200./200.)

In [ ]:
mm1 = 200
mm2 = 200
a0 = 0.0
a1 = -1.0
b0 = 0.3
b1 = 0.5
b = np.arange(a0, a1, (a1-a0)/mm1)
d = np.arange(b0, b1, (b1-b0)/mm2)
B, D = np.meshgrid(b, d)
n, m = B.shape

In [ ]:
fig = plt.figure(figsize=(12, 12))
#ax = Axes3D(fig)
ax = fig.add_subplot(projection='3d')
ax.plot_surface(B, D, spm)
plt.xlabel('$V_1$ (V)', fontsize=22, labelpad=10)
plt.ylabel('$V_2$ (V)', fontsize=22, labelpad=10)
ax.set_zlabel('spm', fontsize=22, rotation=0, labelpad=16)
plt.savefig('spm_' + str(jter) + '.pdf')
plt.show()
plt.close()